```{contents}
```
## **Correction Injection in LangGraph**

**Correction Injection** is a human-in-the-loop control mechanism in LangGraph that allows **external agents (typically humans, sometimes automated validators)** to **modify the execution state mid-run** in order to correct, guide, or override the system’s behavior before execution continues.

It is one of the most important features for building **safe, reliable, production-grade LLM systems**.

---

### **1. Motivation and Intuition**

LLM workflows are probabilistic and error-prone:

| Failure Type      | Example              |
| ----------------- | -------------------- |
| Hallucination     | Fabricated facts     |
| Reasoning errors  | Invalid conclusions  |
| Tool misuse       | Wrong API parameters |
| Policy violations | Unsafe content       |
| Ambiguous goals   | Poor plan selection  |

Instead of restarting the entire workflow, **Correction Injection** enables:

> **Targeted repair of the execution state** without discarding previous progress.

---

### **2. Conceptual Model**

```
LLM → Partial Output → Human Review
                    ↓
              Inject Correction
                    ↓
              Continue Execution
```

This transforms LangGraph from a static automation engine into a **supervised intelligent system**.

---

### **3. Where Correction Injection Lives in LangGraph**

Correction Injection operates at the **State Layer**.

```
Graph Execution
      ↓
 Shared State  ←── Correction Injection
      ↓
 Next Node
```

The injected correction becomes part of the **authoritative state** and immediately influences downstream decisions.

---

### **4. Core Mechanisms**

| Component            | Role                      |
| -------------------- | ------------------------- |
| Interrupt Node       | Pauses execution          |
| State Snapshot       | Exposes full state        |
| Correction Interface | Human edits state         |
| State Merge          | Integrates correction     |
| Resume               | Continues graph execution |

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    question: str
    draft_answer: str
    final_answer: str

def generate(state):
    return {"draft_answer": "Paris is in Germany"}   # incorrect

def finalize(state):
    return {"final_answer": state["draft_answer"]}

builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("finalize", finalize)

builder.set_entry_point("generate")
builder.add_edge("generate", "finalize")
builder.add_edge("finalize", END)

graph = builder.compile()

# Step 1: run until correction point
state = graph.invoke({"question": "Where is Paris?"})

# Step 2: inject correction
state["draft_answer"] = "Paris is in France"

# Step 3: resume
result = graph.invoke(state)
print(result)
```

---

### **6. With Explicit Interrupt Node**

```python
def review(state):
    raise Interrupt(state)

builder.add_node("review", review)
builder.add_edge("generate", "review")
builder.add_edge("review", "finalize")
```

Execution pauses at `review`, allowing safe correction.

---

### **7. Production Workflow**

| Phase     | Description                         |
| --------- | ----------------------------------- |
| Run       | System produces intermediate result |
| Interrupt | Execution halts                     |
| Inspect   | Human reviews state                 |
| Correct   | Modify fields                       |
| Resume    | Continue execution                  |
| Audit     | Log correction                      |

---

### **8. Variants of Correction Injection**

| Variant                  | Use Case                    |
| ------------------------ | --------------------------- |
| Human correction         | Compliance, quality control |
| Rule-based correction    | Automated validation        |
| Tool-assisted correction | External checkers           |
| Multi-agent correction   | Debate / voting             |
| Policy correction        | Safety enforcement          |

---

### **9. Safety & Reliability Benefits**

| Benefit           | Impact                      |
| ----------------- | --------------------------- |
| Error containment | Prevents cascading failures |
| Human oversight   | Regulatory compliance       |
| Explainability    | Transparent decisions       |
| Cost reduction    | Avoids full reruns          |
| Trust             | Enterprise adoption         |

---

### **10. Mental Model**

Correction Injection turns LangGraph into:

> **A controllable intelligent system rather than an uncontrolled generator.**

It enables **real-world AI operations** where **humans remain in command of critical decisions**.

### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt

# -----------------------------
# 1. Define State
# -----------------------------
class State(TypedDict):
    question: str
    draft_answer: str
    final_answer: str

# -----------------------------
# 2. Nodes
# -----------------------------
def generate(state: State):
    print("\nLLM produced:", "Paris is in Germany")
    return {"draft_answer": "Paris is in Germany"}  # Wrong on purpose

def review(state: State):
    # Pause execution and expose state for correction
    interrupt(state)

def finalize(state: State):
    return {"final_answer": state["draft_answer"]}

# -----------------------------
# 3. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("review", review)
builder.add_node("finalize", finalize)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")
builder.add_edge("review", "finalize")
builder.add_edge("finalize", END)

graph = builder.compile()

# -----------------------------
# 4. Run with Correction Injection
# -----------------------------
print("\n--- First Run (Will Interrupt) ---")
state = graph.invoke({"question": "Where is Paris?"})

print("\nState before correction:", state)

# ---- HUMAN CORRECTION ----
state["draft_answer"] = "Paris is in France"
print("\nHuman injected correction!")

print("\n--- Resuming Execution ---")
result = graph.invoke(state)

print("\nFinal result:", result)



--- First Run (Will Interrupt) ---

LLM produced: Paris is in Germany

State before correction: {'question': 'Where is Paris?', 'draft_answer': 'Paris is in Germany', '__interrupt__': [Interrupt(value={'question': 'Where is Paris?', 'draft_answer': 'Paris is in Germany'}, id='53e21a818be5a9f30a0cc7dc80f05736')]}

Human injected correction!

--- Resuming Execution ---

LLM produced: Paris is in Germany

Final result: {'question': 'Where is Paris?', 'draft_answer': 'Paris is in Germany', '__interrupt__': [Interrupt(value={'question': 'Where is Paris?', 'draft_answer': 'Paris is in Germany'}, id='035aa3b4101ef7c08eb5d7cca2330e22')]}
